## Evaluating CrewAI's `crew` (end-to-end)

In this notebook we will demonstrate how you can run evaluations on crews using datasets from Confident AI and DeepEval's dataset iterator.

### Install dependencies:

In [ ]:
!pip install -U deepeval -U crewai ipywidgets --quiet

### Set your OpenAI API key:

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

### Create a crew:

This is a simple crew with a single agent and a single task.

In [ ]:
from crewai import Task, Crew, Agent

agent = Agent(
    role="Consultant",
    goal="Write clear, concise explanation.",
    backstory="An expert consultant with a keen eye for software trends.",
)

task = Task(
    description="Explain the given topic: {topic}",
    expected_output="A clear and concise explanation.",
    agent=agent,
)

crew = Crew(agents=[agent], tasks=[task])

result = crew.kickoff(
    inputs={"topic": "What is the biggest open source database?"}
)
print(result)

### Evaluate the agent

To evaluate CrewAI's `crew`:

1. Instrument the application (using `from deepeval.integrations.crewai import instrument_crewai`)
2. Supply metrics to `kickoff`.


> (Pro Tip) View your Agent's trace and publish test runs on [Confident AI](https://www.confident-ai.com/). Apart from this you get an in-house dataset editor and more advaced tools to monitor and enventually improve your Agent's performance. Get your API key from [here](https://app.confident-ai.com/)


In [ ]:
os.environ["CONFIDENT_API_KEY"] = "<your-confident-api-key>"

In [ ]:
from deepeval.integrations.crewai import instrument_crewai

instrument_crewai()

### Using a dataset from Confident AI:

For demo purposes, we will use a public dataset from Confident AI. You can use your own dataset as well. Refer to the [docs](https://deepeval.com/docs/evaluation-end-to-end-llm-evals#setup-your-test-environment) to learn more about how to create your own dataset.

In [ ]:
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset()
dataset.pull(alias="topic_agent_queries", public=True)

### Run evaluations:

We will use the `AnswerRelevancyMetric` to evaluate the crew. Dataset iterator will yield golden examples from the dataset.

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric

for golden in dataset.evals_iterator():
    result = crew.kickoff(
        inputs={"topic": golden.input}, metrics=[AnswerRelevancyMetric()]
    )

Congratulation! You have just evaluated your first CrewAI's `crew` using Deepeval. Try changing Hyperparameters, Agents, Tasks, Metrics and see how your agent performs.